In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

from utils import get_loaders
from models import SSLNet

print(torch.cuda.is_available())

True


In [4]:
config = {
    'method': 'ssd',
    'temperature': 0.07,
    'contrast_mode': 'one',
    
    # Training
    'epochs': 1000,
    'lr': 5e-4,
    'weight_decay': 1e-5,
    'patience': 20,
    'min_delta': 1e-8,
    'sched_patience': 5,
    'sched_factor': 0.5,
    
    # Network
    'dims' : [30, 16, 8],
    'drop': 0.1,
    'norm': True,
    'activation': 'ReLU',
    
    # Dataset
    'val_split': 0.1,
    'test_split': 0.1,
    'batch_size': 1024,
    
    # Utility
    'seed': 15,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'data_path': './Data/creditcard.csv',
}

In [5]:
train_loader, val_loader, test_loader = get_loaders(config['data_path'], config['val_split'], config['test_split'], config['seed'], config['batch_size'], config['method'])

(230693, 31) (25633, 31) (28481, 31)


In [6]:
df = pd.read_csv('./Data/creditcard.csv')
for i in ['Time', 'Amount']:
    df.loc[:, i] = (df[i] - df[i].mean()) / df[i].std()

train, test = train_test_split(df, test_size=config['test_split'], stratify=df['Class'],
                               random_state=config['seed'])
train, val = train_test_split(train, test_size=config['val_split'], stratify=train['Class'],
                              random_state=config['seed'])

print(train.shape, val.shape, test.shape)

x = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

x_val = val.iloc[:, :-1].values
y_val = val.iloc[:, -1].values

x_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values

if config['method'] == 'ssd':
    x_pos = x + np.random.normal(0, 0.05, x.shape)
    train = torch.utils.data.TensorDataset(torch.tensor(x, dtype=torch.float32), 
                                           torch.tensor(x_pos, dtype=torch.float32), 
                                           torch.tensor(y, dtype=torch.float32)
                                           )

else:
    train = torch.utils.data.TensorDataset(torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
val = torch.utils.data.TensorDataset(torch.tensor(x_val, dtype=torch.float32),
                                     torch.tensor(y_val, dtype=torch.float32))

(230693, 31) (25633, 31) (28481, 31)


In [7]:
model = SSLNet(config)
print(model)

SSLNet(
  (embeddings): Sequential(
    (0): Linear(in_features=30, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)


In [14]:
x = torch.tensor(x, dtype=torch.float32)
x_pos = torch.tensor(x_pos, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [46]:
contrast_mode = 'one'
temperature = 0.07

In [17]:
images = torch.cat([x, x_pos], dim=0)
batch_size = y.shape[0]

features = model(images)
f1, f2 = torch.split(features, [batch_size, batch_size], dim=0)
features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
loss = criterion(features)

In [77]:
f1.shape

torch.Size([1024, 8])

In [78]:
lists = []

In [83]:
lists.append(f1[3:6])

In [86]:
torch.concat(lists).shape

torch.Size([6, 8])